# Welcome to ProgPy's basic example notebook!

This example extends the "basic example" to perform a state estimation and prediction with uncertainty given a more complicated model. Models, state estimators, and predictors can be switched out. See [documentation](nasa.github.io/progpy) for description of options.



Method: An instance of the BatteryCircuit model in progpy is created, and the prediction process is achieved in three steps:
    
1.  State estimation of the current state is performed using a chosen state_estimator, and samples are drawn from this estimate
2.  Prediction of future states (with uncertainty) and the times at which the event threshold will be reached
3.  Metrics tools are used to further investigate the results of prediction

Results: 
1. Predicted future values (inputs, states, outputs, event_states) with uncertainty from prediction
2. Time event is predicted to occur (with uncertainty)
3. Various prediction metrics
4. Figures illustrating results

#### Importing Necessary Modules

For our demonstration, we'll have to import four modules from progpy. The first module is the model we'll be using to preform Prognostics, the second being the state estimate we'll use to estimate the current state of the system. The third module being the predictor we'll use to predict the future states of the system, and the fourth module is a module to calculate probability of success from the prediction metrics.

In [ ]:
from prog_models.models import BatteryCircuit as Battery

from prog_algs.state_estimators import ParticleFilter as StateEstimator

from prog_algs.predictors import MonteCarlo as Predictor

from prog_algs.metrics import prob_success

### Step 1: Setup model & future loading

Firstly, we'll begin by defining the measurement noise for the outputs our system. We'll state that two specific parameters in our model, the `t` and `v` parameters, are the only parameters that are subject to this noise. We'll also define the noise for each of these parameters to be 2 and 0.02 respectively.

Then, we can initialize our Battery model to have a process_noise of 0.25 for every state.

In [ ]:
# Measurement noise
R_vars = {
    't': 2,
    'v': 0.02
}

batt = Battery(process_noise = 0.25, measurement_noise = R_vars)

For more information on what this particular Battery Model is, please refer to our [BatteryCircuit Documentation](https://nasa.github.io/progpy/api_ref/prog_models/IncludedModels.html#prog_models.models.BatteryCircuit).

For more general information regarding what are Prognostics Models are, please refer to our [prog_models Guide](https://nasa.github.io/progpy/api_ref/prog_models/index.html).

Now, let's create the input containers outside of the function accelerates prediction.

In [ ]:
loads = [
    batt.InputContainer({'i': 2}),
    batt.InputContainer({'i': 1}),
    batt.InputContainer({'i': 4}),
    batt.InputContainer({'i': 2}),
    batt.InputContainer({'i': 3})
]

Finally, let's define the `future_loading()` function and initialize the state of our battery model to `initial_state`.

In [ ]:
def future_loading(t, x = None):
    # Variable (piece-wise) future loading scheme 
    if (t < 600):
        return loads[0]
    elif (t < 900):
        return loads[1]
    elif (t < 1800):
        return loads[2]
    elif (t < 3000):
        return loads[3]
    return loads[-1]

initial_state = batt.initialize()

### Step 2: Demonstrating State Estimator

Let's begin by defining the StateEstimator object by passing in the Battery Model and its initial state.

In [ ]:
filt = StateEstimator(batt, initial_state)

Before we make an estimations of future states, let's plot the current state of the system.

In [ ]:
print("Prior State:", filt.x.mean)
print('\tSOC: ', batt.event_state(filt.x.mean)['EOD'])
fig = filt.x.plot_scatter(label='prior')

Now, let's preform the state estimation step and plot the posterior results!

In [ ]:
# State Estimation Step
example_measurements = batt.OutputContainer({'t': 32.2, 'v': 3.915})
t = 0.1
u = future_loading(t)
filt.estimate(t, u, example_measurements)

In [ ]:
# Printing Results
print("\nPosterior State:", filt.x.mean)
print('\tSOC: ', batt.event_state(filt.x.mean)['EOD'])
filt.x.plot_scatter(fig=fig, label='posterior')  # Add posterior state to figure from prior state

For more information on State Estimation, please refer to our [State Estimation Guide](https://nasa.github.io/progpy/prog_algs_guide.html#state-estimation).

__`Note`__: in a prognostic application the above state estimation step would be repeated each time there is new data. Here we're doing one step to demonstrate how the state estimator is used

### Step 3: Demonstrating Predictor

To demonstrate the predictor, we must first setup the predictor object. We can do this by passing in the Battery Model.

In our example, we are using are using the MonteCarlo method to make a prediction.

In [ ]:
mc = Predictor(batt)

Now, let's preform a prediction! We can call the `predict()` method on our predictor object and pass in the number of samples we want to use to make our prediction. In this example, we'll use 25 samples, a step size of 0.1, and a save frequency of 100.

In [ ]:
NUM_SAMPLES = 25
STEP_SIZE = 0.1
SAVE_FREQ = 100  # How often to save results
mc_results = mc.predict(filt.x, future_loading, n_samples = NUM_SAMPLES, dt=STEP_SIZE, save_freq = SAVE_FREQ)
print('ToE', mc_results.time_of_event.mean)

For more information on the `predict()` function, please refer to our [Prediction Documentation](https://nasa.github.io/progpy/prog_algs_guide.html#prediction).

Let's analyze the results of the prediction!

__`Note`__: The results of a sample-based prediction can be accessed by sample, e.g., `predictor.results[0]` is the first sample. The results of a non-sample-based prediction can be accessed by calling the `predictor.results` attribute.

In [ ]:
from prog_algs.predictors import UnweightedSamplesPrediction
if isinstance(mc_results, UnweightedSamplesPrediction):
    states_sample_1 = mc_results.states[1]

Note, that the states_sample_1[n] corresponds to times[n] for the first sample

You can also access a state distribution at a specific time using the `snapshot()` function.

In [ ]:
states_time_1 = mc_results.states.snapshot(1)

Now you have all the samples corresponding to times[1]!

For more information on analyzing results of predictors, please refer to our [Analyzing Results Documentation](https://nasa.github.io/progpy/prog_algs_guide.html#analyzing-results).

### Reviewing Results

Now that we have our results, let's plot the results of our prediction!

In [ ]:
print('Results: ')
for i, time in enumerate(mc_results.times):
    print('\nt = {}'.format(time))
    print('\tu = {}'.format(mc_results.inputs.snapshot(i).mean))
    print('\tx = {}'.format(mc_results.states.snapshot(i).mean))
    print('\tz = {}'.format(mc_results.outputs.snapshot(i).mean))
    print('\tevent state = {}'.format(mc_results.event_states.snapshot(i).mean))

You can also access the final state (of type UncertainData), like so:

In [ ]:
final_state = mc_results.time_of_event.final_state
print('Final state @EOD: ', final_state['EOD'].mean)

You can also use the metrics package to generate some useful metrics on the result of a prediction

In [ ]:
print("\nEOD Prediction Metrics")
print('\tPortion between 3005.2 and 3005.6: ', mc_results.time_of_event.percentage_in_bounds([3005.2, 3005.6]))
print('\tAssuming ground truth 3002.25: ', mc_results.time_of_event.metrics(ground_truth=3005.25))
print('\tP(Success) if mission ends at 3002.25: ', prob_success(mc_results.time_of_event, 3005.25))

Finally, we can also plot the state transition! Here we will plot the states at t0, 25% to ToE, 50% to ToE, 75% to ToE, and ToE

In [ ]:
fig = mc_results.states.snapshot(0).plot_scatter(label = "t={} s".format(int(mc_results.times[0])))  # 0
quarter_index = int(len(mc_results.times)/4)
mc_results.states.snapshot(quarter_index).plot_scatter(fig = fig, label = "t={} s".format(int(mc_results.times[quarter_index])))  # 25%
mc_results.states.snapshot(quarter_index*2).plot_scatter(fig = fig, label = "t={} s".format(int(mc_results.times[quarter_index*2])))  # 50%
mc_results.states.snapshot(quarter_index*3).plot_scatter(fig = fig, label = "t={} s".format(int(mc_results.times[quarter_index*3])))  # 75%
mc_results.states.snapshot(-1).plot_scatter(fig = fig, label = "t={} s".format(int(mc_results.times[-1])))  # 100%

mc_results.time_of_event.plot_hist()
